In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
data = pd.read_csv('../../dataset (missing + split)/trainFilled.csv', skip_blank_lines=True)